In [1]:
from tqdm import tqdm
from datetime import datetime
from modules.utils import *
import os

## Input job search keyword

In [2]:
keyword = 'data analyst'
pages = 10

# with open('config.yaml', 'r') as f:
#     config = yaml.safe_load(f)
#     keyword_list = config['job_titles']
#     pages = config['pages']

In [ ]:
# links = get_job_links(keyword, start_page=0, pages=pages)
links, main_soup = get_job_links_selenium(keyword, pages=pages, debug=True)
main = {}
soups = {}
for index, link in tqdm(enumerate(links), total = len(links), dynamic_ncols =True):
    main[index], soups[index] = get_job_info(link, index, return_soup=True)

In [16]:
# links
# len(main)
# soups[88]

In [19]:
df = process_df(main, remove_nulls=True, remove_duplicates=True)
print(len(df))
df.head()

108


,date_logged,company,job_title,level,job_type,experience,spark,degree,descriptions,industry1,industry2,link
job_id,,,,,,,,,,,,
3786883030,2023-12-27,ABBVIE OPERATIONS SINGAPORE PTE. LTD.,Business Data Analyst,Mid-Senior level,Full-time,,,•Bachelor’s degree with 3 to 5 years of manufa...,\nJob Overview\nThis position is responsible f...,Information Technology,NaN,https://sg.linkedin.com/jobs/view/business-dat...
3780542368,2023-12-27,ARTA TECH PTE. LTD.,Data Analyst,Entry level,Full-time,•Bachelor's degree. MBA is a plus.Minimum 3 ye...,,•Bachelor's degree. MBA is a plus.Minimum 3 ye...,Arta Finance is a dynamic fintech star...,Information Technology,NaN,https://sg.linkedin.com/jobs/view/data-analyst...
3793406045,2023-12-27,ATTRIBUTE DATA PTE. LTD.,Data Analyst,Entry level,Full-time,•Take a lead role in partnering with our clien...,,,\nPrimary Responsibilities\n•Take a lead role ...,Information Technology,NaN,https://sg.linkedin.com/jobs/view/data-analyst...
3744539391,2023-12-27,Adecco,Junior/Senior Data Analyst | Power BI | Immedi...,Mid-Senior level,Full-time,•Holds a Degree in Computer Science/ Business ...,,,Adecco is partnering with a Global com...,Information Technology,Staffing and Recruiting,https://sg.linkedin.com/jobs/view/junior-senio...
3752083944,2023-12-27,Agile Talent,Project Control Engineer (Data Analyst),Associate,Contract,"•Bachelor degree in Engineering, Accounting, B...",,"•Bachelor degree in Engineering, Accounting, B...",\nResponsibilities\n•Project Controls Engineer...,Management and Manufacturing,Staffing and Recruiting,https://sg.linkedin.com/jobs/view/project-cont...


### By saving and compiling all the intermediate files

In [26]:
most_recent_file = f"{keyword}_{datetime.now().strftime('%Y-%m-%d-%H%M')}.xlsx"
df.to_excel(f"ignore/{most_recent_file}", engine='xlsxwriter')
most_recent_filepaths = [file for file in os.listdir('ignore') if datetime.now().strftime('%Y-%m-%d') in file]
most_recent_filepaths

['data analyst_2023-12-27-2331.xlsx']

In [27]:
# main_df = update_main('files/MAIN_2023-12-25.xlsx', most_recent_filepaths)
main_df = update_main('files/MAIN_2023-12-27.xlsx', [df])

In [28]:
main_df.to_excel(f"ignore/MAIN_{datetime.now().strftime('%Y-%m-%d')}.xlsx", engine='xlsxwriter')